In [ ]:
### DOWNLOADS/IMPORTS

! apt-get install poppler-utils
! sudo apt install tesseract-ocr
! sudo apt-get install tesseract-ocr
! pip install pdf2image
! pip install pytesseract
! wget "https://nss.org.sg/report/8baffd7a-9Avi_2005_19_1.pdf" # DONE 4:36
! wget "https://nss.org.sg/report/1eebac90-8Avi_2005_19_2.pdf" # DONE 4:20
! wget "https://nss.org.sg/report/481b0c11-eAvi_2005_19_3.pdf" # DONE 3:17
! wget "https://nss.org.sg/report/3d428c84-fAvi_2006_19_4.pdf" # DONE 5:26
! wget "https://nss.org.sg/report/bdd6d2d2-dAvi_2006_20_1.pdf" # DONE 3:19
! wget "https://nss.org.sg/report/7f6f74f5-7Avi_2000_14_1.pdf" # DONE 2:24
! pip install word2number
% load_ext google.colab.data_table

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('maxent_treebank_pos_tagger') 

from google.colab import drive
import pandas as pd
import pickle

drive.mount('/content/gdrive')
df_in = pd.read_csv("/content/gdrive/My Drive/df (10).csv")
datafile = pd.read_csv("/content/gdrive/My Drive/ebird-checklist.csv", encoding= 'unicode_escape')
cachedsci = pickle.load(open("/content/gdrive/My Drive/cachedsci.pkl", "rb"))
cachedeng = pickle.load(open("/content/gdrive/My Drive/cachedeng.pkl", "rb"))
cachedpicks = pickle.load(open("/content/gdrive/My Drive/cachedpicks.pkl", "rb"))
contribs = pickle.load(open("/content/gdrive/My Drive/contributors.pkl", "rb"))
% cd ..

from pdf2image import convert_from_path, convert_from_bytes
import cv2
import pytesseract
import numpy as np
import re
from google.colab.patches import cv2_imshow
from tqdm.notebook import tqdm
from functools import partial

import pprint
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk
import pandas as pd
import nltk.tag, nltk.data
import copy
from word2number import w2n

from google.colab import files
from datetime import datetime
import editdistance
from tqdm.notebook import tqdm
from IPython.display import display, HTML, clear_output
import time
import pickle
from google.colab import data_table

In [6]:
### OCR

tqdm = partial(tqdm, position=0, leave=True)

images = convert_from_bytes(open('/7f6f74f5-7Avi_2000_14_1.pdf', 'rb').read())

text = ''

contains = re.compile(r'((?:[A-Z][a-z]+ )+\(([A-Z ]+)\))')

nu = 0

for i in tqdm(images[2:24]): # include the contributors page!
  open_cv_image = np.array(i) 
  img = open_cv_image[:, :, ::-1].copy()
  img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  # gray = cv2.medianBlur(gray, 3)
  gray, img_bin = cv2.threshold(gray,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  gray = cv2.bitwise_not(img_bin)

  kernel = np.ones((2, 1), np.uint8)
  img = cv2.erode(gray, kernel, iterations=1)
  img = cv2.dilate(img, kernel, iterations=1)
  tt = pytesseract.image_to_string(cv2.bitwise_not(img))
  if nu != 0:
    text += (tt.replace("/\n","/") + "\n\n")
  else: 
    contributors = (tt.replace("\n"," "))
  nu += 1

contribs2 = contains.findall(contributors)
contribs2 = [[b.replace('(', '').replace(')', ''), a.replace(f'({b})', '')] for (a, b) in contribs2]
contribs2[-1][1] = contribs2[-1][1].replace(' and ', '')
contribs2 = dict(contribs2)

for k in contribs2.keys():
  contribs[k] = contribs2[k]

con = open("content/contributors.pkl", "wb")
pickle.dump(contribs, con)
con.close()

In [ ]:
### DATA PT. 1

cachedpicks = pickle.load(open("/content/gdrive/My Drive/cachedpicks.pkl", "rb"))

res2005 = re.compile(r'^([A-Z -]){5,}.*?[A-Z]?[a-z ]*$')
res = re.compile(r"^[A-Z]([A-Z]?[a-z]+[ '-])*[/a-z]+$")
res2 = re.compile(r"^[A-Z]['/A-Za-z -]{20,}")
splitter = re.compile(r'([({]a?l?l?[A-Z/ ]*[)/}]|\(sev obs\)|\(both by [A-Z/]+\)|\(both [A-Z/]+\))')
date = re.compile(r'([0-9]{1,2} ?/ ?[0-9]{1,2})|(Multidate\[.*\])')
address_re = re.compile(r'( [A-Z][a-z]+)+ ((Road)|(Lane)|(Street)|(Avenue)) [0-9]+')
volume = re.compile(r'Volume \d+ No \d+')
rang = re.compile(r'([0-9]{1,2} ?/ ?[0-9]{1,2}) to ([0-9]{1,2} ?/ ?[0-9]{1,2})')
rang2 = re.compile(r'([0-9]{1,2})-([0-9]{1,2} ?/ ?([0-9]{1,2}))')
multi = re.compile(r'((([0-9]{1,2} ?/ ?[0-9]{1,2}),? a?n?d? ?){2,})')
multisub = re.compile(r'Multidate\[(.*)\]')

def flatten(l):
  return [item for sublist in l for item in sublist]

def address(string, address_re):
  try:
    addr = address_re.search(string).group()
    return ([addr], [j for j in addr.split(' ') if j.isdigit()])
  except AttributeError:
    return [[], []]

class Sighting:
  def __init__(self, species, date, count, person, location, string=None):
    self.person = person
    self.date = date
    self.species = species
    self.count = count
    self.string = string
    self.location = location
    self.data = [species, date, count, person, location, string]
  def __str__(self):
    return f"SPECIES: {self.species}\nDATE: {self.date}\nCOUNT: {self.count}\nPERSON: {self.person}\nLOCATION: {self.location}\nCOMMENTS: {self.string}"
  def join(self, other):
    data2 = zip(self.data, other.data)
    datanew = []
    n0 = 0
    for a, b in data2:
      if n0 == 3:
        n0 += 1
        if a in ['', 0] and b not in ['', 0]:
          datanew.append([b, b])
        else:
          datanew.append([a, b])
        continue
      if a in ['', 0] and b not in ['', 0]:
        datanew.append([b, b])
      elif a not in ['', 0] and b in ['', 0]:
        datanew.append([a, a])
      else:
        datanew.append([a, b])
      n0 += 1
    return (Sighting(*[a for a, b in datanew]), Sighting(*[b for a, b in datanew]))

def make_df(sightings):
  d = {
      "Species": [i.species for i in sightings],
      "Date": [i.date for i in sightings],
      "Count": [i.count for i in sightings],
      "Observer": [i.person for i in sightings],
      "Location": [i.location for i in sightings],
      #"Comments": [i.string for i in sightings]
  }
  df = pd.DataFrame.from_dict(d)
  return df

def get_multidates(tex):
  out = flatten(rang.findall(tex))
  out2 = flatten(rang2.findall(tex))
  out3 = [i for i in flatten(multi.findall(tex)) if i != 'and' and i != '']
  if out !=  []:
    return rang.sub(r"Multidate[\1,\2]", tex)
  if out2 != []:
    return rang2.sub(r"Multidate[\1/\3,\2]", tex)
  if out3 != []:
    txt = multi.sub(r"Multidate[\1] ", tex)
    for it in multisub.findall(txt):
      txt = re.sub(',+', ',', txt.replace(it, it.replace(" ", ",").replace('and', ''))).replace(',]', ']')
    return txt
  return tex

def look(arr, item, prev = 0):
  for i in range(prev+1, len(arr)):
    if item in arr[i]:
      return i - 1
  print(arr, item)

spdata = {}
spdata2 = []

latest = ''

for t in text.split("\n"):
  if re.match(res, t.strip())!=None or re.match(res, t.strip()[:-5])!=None:
    if t not in spdata.keys():
      spdata[t] = ''
    latest = t
  elif re.match(res2, t.strip())!=None:
    ask = input("Is this a species? " + t.strip() + "\n").strip()
    if ask.lower() != '':
      if t not in spdata.keys():
        spdata[t] = ''
      latest = t
    elif latest != '':
      spdata[latest] += (' ' + t)
  elif latest != '':
    spdata[latest] += (' ' + t)

n = 0

for k, iv in list(spdata.items()):
  n += 1
  #if len(date.findall(i)) <= len(splitter.findall(i)):
  s = splitter.split(iv)
  i = get_multidates(iv)

  dates = [k for k in flatten(date.findall(i)) if k != '']
  
  li = list(zip(s[::2], s[1::2], dates))
  li = [(a.replace(c,''), b, c) for a, b, c in li ]

  # s2 = date.split(i)
  # dates2 = (date.findall(i))
  # obs2 = splitter.findall(i)
  # li2 = list(zip(s2, obs2, dates2))
  # li2 = [(a.replace(c,''), b,  c) for a, b, c in li2 ]
  
  # prev = 0
  # numb = 0

  # for i in obs2:
  #   index = look(obs2, i, prev)
  #   li2[index].append(obs2[numb])
  #   numb += 1

  # li2 = [[s2_val, obs2_val, dates2_val] for (s2_val, dates2_val, ovs2_val) in li2]

  # if len(obs2) < len(dates):
  #   print(s2, dates2, obs2)
  #   li = copy.deepcopy(li2)

  for string1, person, d in li:

    string = volume.sub('', string1)
    matched, ignore = address(string, address_re)
    matched = [i1.strip() for i1 in matched]
    if "Track 270" in string:
      matched.append("Track 270")
      ignore.append("270")
    original = copy.deepcopy(string)
    string = string.rstrip(', ').lstrip(', ')
    for repl in ["a pair", "1 pair", "another pair", "pair", "1 male and 1 female", "a male and a female", "an adult and a juvenile", "1 seen and 1 heard"]:
      string = re.compile(repl, re.IGNORECASE).sub('2', string)
    for repl in ["several", "a number"]:
      string = re.compile(repl, re.IGNORECASE).sub('1111', string)
    string = string.replace("Singapore Avifauna", '')
    string = string.replace("Report", '')
    for repl in ["a juvenile", "a female", "a male", "a summer male", "another", "\|", "an immature", "an adult"]:
      string = re.compile(repl, re.IGNORECASE).sub('1', string)
    string = re.compile("including \d", re.IGNORECASE).sub('', string)

    if all([word in ["and", ",", ''] for word in string.split(" ")]) or len(string) == 0:
      spdata2.append(Sighting(k, d, 0, person, '', string=string))
      continue
    location = matched
    count = []
    wtk = word_tokenize(string)
    pos_tag1 = pos_tag(wtk)
    chunk = ne_chunk(pos_tag1)
    for ix in chunk.subtrees(filter=lambda x: x.label() in ["PERSON", "ORGANIZATION", "FACILITY", "LOCATION"]):
      location.append(" ".join([a for a, b in ix]))
    if len(location) == 0:
      for ix in wtk:
        if ix[0].isupper():
          location.append(ix)
      location = [" ".join(location)]
    location = [l for l in location if l not in ["Sungei Puaka", "Kg Melayu", "Nesting Report", "Ficus", "Trail", "African Tulip Tree"]]
    for ix in chunk:
      if len(ix) > 1 and (ix[1] == "CD") and '/' not in ix[0]:
        count.append(ix[0])
    for num in ignore:
      count.remove(num)
    string = re.compile("[0-9]+ ?meters", re.IGNORECASE).sub('', string)
    string = re.compile("[0-9]+ ?metres", re.IGNORECASE).sub('', string)
    if len(count) != 1:
      while True:
        inp = input(f"COUNT for {string.strip()}      {n}/{len(spdata.items())}\n").strip()
        if inp == 'skip': 
          break
        try: 
          if inp == '':
            count = 0
            break
          count1 = int(inp)
          break
        except:
          if inp == 'more':
            print(k, i)
          continue
    else:
      inp = count[0].replace('.', '')
    if count != 0 and inp != 'skip': 
      try:
        count = int(inp)
      except ValueError:
        try:
          count = w2n.word_to_num(inp)
        except ValueError:
          while True:
            inp = input(f"COUNT for {string.strip()}      {n}/{len(spdata.items())}\n").strip()
            if inp == 'skip': 
              break
            try: 
              if inp == '':
                count = 0
                break
              count1 = int(inp)
              break
            except:
              if inp == 'more':
                print(k, i)
              continue
    for place in location:
      if len(matched) > 0 and place in matched[0] and place != matched[0] and place != '':
        location.remove(place) # remove subsets

    location = list(set(location))
    location = [loc for loc in location if loc != 'Multidate']
    if len(location) > 1:
      try:
        l1 = copy.deepcopy(location)
        location = cachedpicks[tuple(location)]
      except KeyError:
        while True:
          inp = input(f"LOCATION for {string.strip()}; options: {location}      {n}/{len(spdata.items())}\n").strip()
          if inp == 'skip': 
            break
          try: 
            if inp == '':
              location = ''
              break
            location1 = location[int(inp)]
            break
          except:
            if inp == 'more':
              print(k, i)
            continue
    else:
      if inp != 'skip':
        inp = 0
    try:
      if location != '' and inp != 'skip' and type(location) == list:
        cachedpicks[tuple(location)] = location[int(inp)]
        location = location[int(inp)]
    except ValueError:
      location = " ".join([ location[i] for i in map(int, inp.split(',')) ])
    if inp != 'skip':
      if type(location) == list:
        location = location[0]
      sight = Sighting(k, d, count, copy.deepcopy(person), copy.deepcopy(location), string=string)
      spdata2.append(sight)
    else:
      print("skipped this one")

for x in range(1, len(spdata2)):
  spdata2[x], spdata2[x-1] = spdata2[x].join(spdata2[x-1])

a_file3 = open("content/cachedpicks.pkl", "wb")
pickle.dump(cachedpicks, a_file3)
a_file3.close()

In [ ]:
made

In [158]:
### DATA PT. 2
from tqdm.notebook import tqdm

tqdm.pandas()

# cachedeng = {k: list(datafile.loc[datafile["scientific name"] == v]["English name"])[0] for k, v in cachedsci.items()}
# print(cachedeng)

# a_file = open("cachedeng.pkl", "wb")
# pickle.dump(cachedeng, a_file)
# a_file.close()

overrides = {
    "Lonchura molucca": "Lonchura atricapilla",
    "Black-faced Munia": "Black-headed Munia",
    "Nilgiri Flowerpecker": "Plain Flowerpecker",
    "Dicaeum concolor": "Dicaeum minullum",
    "Collared Scops-Owl": "Sunda Scops-Owl",
    "Otus lettia": "Otus lempiji"
}

cachedsci = {k: v if v not in overrides.keys() else overrides[v] for (k, v) in cachedsci.items()}
cachedeng = {k: v if v not in overrides.keys() else overrides[v] for (k, v) in cachedeng.items()}

done = 0

def get(spname):
  global done
  datafile2 = datafile.copy(deep=True)
  text = spname.lower()
  datafile2["Distance"] = datafile2["Pair"].apply(lambda i: editdistance.eval(text, i))
  datafile2 = datafile2.sort_values(by='Distance')
  vsci = list(datafile2["scientific name"])[0]
  veng = list(datafile2["English name"])[0]
  score = list(datafile2["Distance"])[0]
  if score > 5:
    try:
      print(text, done)
      n = 3
      display(datafile2.head(3))
      inpu = input("Confirm?").strip()
      while inpu == 'more':
        n += 5
        display(datafile2.head(n))
        inpu = input("Confirm?").strip()
      if inpu != '':
        vsci = list(datafile2["scientific name"])[int(inpu)]
        veng = list(datafile2["English name"])[int(inpu)]
        inpu2 = input("Confirm again").strip()
        if inpu != '':
          vsci = list(datafile2["scientific name"])[int(inpu2)]
          veng = list(datafile2["English name"])[int(inpu2)]
    except ValueError:
      text2 = input("Species name").strip()
      datafile2["Distance"] = datafile2["Pair"].apply(lambda i: editdistance.eval(text2, i))
      datafile2 = datafile2.sort_values(by='Distance')
      vsci = list(datafile2["scientific name"])[0]
      veng = list(datafile2["English name"])[0]
      score = list(datafile2["Distance"])[0]
      if score > 5:
          print(text2)
          n = 3
          display(datafile2.head(3))
          inpu = input("Confirm?").strip()
          while inpu == 'more':
            n += 5
            display(datafile2.head(n))
            inpu = input("Confirm?").strip()
          if inpu != '':
            vsci = list(datafile2["scientific name"])[int(inpu)]
            veng = list(datafile2["English name"])[int(inpu)]
            inpu2 = input("Confirm again").strip()
            if inpu != '':
              vsci = list(datafile2["scientific name"])[int(inpu2)]
              veng = list(datafile2["English name"])[int(inpu2)]
        
  cachedsci[text] = vsci
  cachedeng[text] = veng
  clear_output()
  return (vsci, veng)

def expand(obser):
  if type(obser) == str:
    obser = obser[1:-1]
  else:
    return None
  text = []
  for o in obser.split('/'):
    try:
      text.append(contribs[o.replace(' ', '')].strip())
    except:
      text.append(o)
  return '/'.join(text)

def optimum(spname, scieng):
  global done
  done += 1
  global cachedsci, cachedeng
  try:
    if scieng == 'sci':
      return cachedsci[spname.lower()]
    if scieng == 'eng':
      return cachedeng[spname.lower()]
  except KeyError:
    if scieng == 'sci': 
      return get(spname)[0]
    if scieng == 'eng': 
      return get(spname)[1]

made = make_df([i for i in spdata2 if type(i) == Sighting])
made["Comments"] = made["Species"].apply(lambda i: spdata[i].strip())
made["Observer(s)"] = made["Observer"].apply(expand)

df = pd.concat([df_in, made])
# df = df_in

YEAR = 0

def dt(row):
  global YEAR
  d = row["Date"]
  if type(d) == str:
    try:
      if d.count('/') == 1:
        if YEAR == 0:
          YEAR = int(input("What year were the added sightings seen in?\n").strip())
        return datetime(year=YEAR,month=int(d.split("/")[1]),day=int(d.split("/")[0]))
      elif d.count('/') == 2:
        return datetime(year=int(d.split("/")[2]),month=int(d.split("/")[0]),day=int(d.split("/")[1]))
      if d.count('-') == 2:
        return datetime(year=int(d.split("-")[0]),month=int(d.split("-")[1]),day=int(d.split("-")[2]))
    except:
      return None
  else:
    return d


df["Date"] = df.apply(dt,axis=1)

datafile["Pair"] = datafile["Pair"].apply(lambda i: i.lower())
df["English name"] = df["Species"].progress_apply(lambda i: optimum(i, 'eng'))
df["Scientific name"] = df["Species"].apply(lambda i: optimum(i, 'sci'))
df["Count"] = df["Count"].apply(lambda i: "X" if i == 1111 or i == 111 else i)

df = df[~(df["Date"]==None)]

a_file = open("content/cachedsci.pkl", "wb")
pickle.dump(cachedsci, a_file)
a_file.close()

a_file2 = open("content/cachedeng.pkl", "wb")
pickle.dump(cachedeng, a_file2)
a_file2.close()

df = df[[co for co in df.columns if co != "Observers2"]]

df["Count"] = df["Count"].apply(lambda i: i if type(i) == int or str(i).isdigit() else 'X')

df = df.drop_duplicates(subset = [i for i in df.columns if i != "Species comments"])

cols = ["English name", "Scientific name", "Date", "Count", "Observer(s)", "Location", "Comments"]
cols2 = ["Species", "Date", "Count", "Observer(s)", "Location", "Comments"]

df_display = df[cols]

df_display = df_display[~df_display["Date"].isnull()]

def process(item):
  d = item.Date
  try:
    date = d.strftime("%d").lstrip("0") 
  except:
    print(item)
    raise

  month = d.strftime("%m").lstrip("0")
  dynamic_regex = re.compile(date + ' ?/ ?' + month)
  a = []

  for i in item.Comments.split('.'):
    if dynamic_regex.search(i):
      a.append([i.strip(), item.Location in i])

  if len(a) > 1:
    a = [[i, j] for (i, j) in a if j]

  a = " /// ".join([c for c, b in a])
  
  if a == '':
    print(item["English name"] + " was removed.")
    print("More details: ")
    print(item.Location)
    print(item.Comments)
    print(str(item.Date).split(' ')[0] + "\n")
    a = "None"

  item[-1] = a
  return item

def trip(i):
  vol = i.year - 1986
  return f'These sightings are reproduced with permission from Volume {vol} of Singapore Avifauna, which was published originally by Nature Society (Singapore).\
            The original report is available at https://nss.org.sg/report.aspx.'

df_display = df_display.apply(process, axis=1)
df_display = df_display[df_display["Comments"] != "None"]
df_display["Trip comments"] = df_display["Date"].apply(trip) 

df_display = df_display.drop_duplicates()

df_in = df.copy(deep=True)

df[cols2].to_csv("content/df (10).csv", index=False)
# files.download("content/df (10).csv")

df_display.to_csv("content/output.csv", index=False)
# files.download("content/output.csv")

data_table.DataTable(df_display, include_index=False)


Greater Racket-tailed Drongo was removed.
More details: 
Botanic Gardens
1 seen sitting in a nest on an Albizia branch 12 metres above the ground at Dairy Farm Road, 13/4 to 21/4 (LKS). The nest was suspended on a horizontal branch just metres away from a huge hive of honey-bees! Another bird was seen sitting in a nest 10 metres up another albizia at Belukar Track, BTNR, on 21/4 (LKS).  1 at Botanic Gardens, 15/5 (LKS).
2005-04-21

Changeable Hawk-Eagle was removed.
More details: 
Large-billed Crow Island Club Road
1 heard near Sungei Jelutong, Pulau Ubin on 1/5 (DA/NK/LKS/JR). Another heard ar MacRitchie Reservoir, 3/5 (LKS). 1 was seen being mobbed by a Large-billed Crow along Island Club Road, 3/5  (LKS).
2005-05-03

White-breasted Waterhen was removed.
More details: 
Marina City Park
1 adult seen with a full-grown chick at SBWR, 7/] (DA/LKS) and 1 independent immature at Marina City  Park, 13/1 (LKS).  1 chick seen with an adult at Botanic Gardens, 4/2 (LKS) and 26/2 (LKS). Anothe

,English name,Scientific name,Date,Count,Observer(s),Location,Comments,Trip comments
0,Red Junglefowl,Gallus gallus,2005-04-24,2,Lim Kim Seng/Bruce Wedderburn,Pulau Ubin,2 females seen foraging in abandoned rubber pl...,These sightings are reproduced with permission...
1,Red Junglefowl,Gallus gallus,2005-04-29,4,Lim Kim Seng,Pulau Ubin,Another male and 3 females were seen in mangro...,These sightings are reproduced with permission...
2,Red Junglefowl,Gallus gallus,2005-04-24,1,Lim Kim Seng/Bruce Wedderburn,SBWR,1 male seen around the canteen at SBWR visitor...,These sightings are reproduced with permission...
3,Red Junglefowl,Gallus gallus,2005-05-01,3,Doreen Ang/Nessie Khoo/Lim Kim Seng/Jane Rogers,Pulau Ubin,"1 male and 2 females seen on Pulau Ubin, 1/5 (...",These sightings are reproduced with permission...
4,Wandering Whistling-Duck,Dendrocygna arcuata,2005-04-24,1,Lim Kim Seng/Bruce Wedderburn,Symphony Lake,1 seen with Lesser Whistling-ducks at Symphony...,These sightings are reproduced with permission...
...,...,...,...,...,...,...,...,...
360,Spectacled Bulbul,Rubigula erythropthalmos,2000-03-05,2,Chris Artuoso,Panti Forest,"At Panti Forest (Bunker Track), 2 chicks been ...",These sightings are reproduced with permission...
361,Rufous-fronted Babbler,Cyanoderma rufifrons,2000-03-14,1,Lim Kim Chuah,Panti Forest,"| heard at Panti Forest (Bunker Track), 14/3 (...",These sightings are reproduced with permission...
362,Horsfield's Babbler,Turdinus sepiarius,2000-03-05,1,Chris Artuoso,Panti Forest,1 very vocal bird at Panti Forest (Bunker Trac...,These sightings are reproduced with permission...
363,Yellow-vented Flowerpecker,Dicaeum chrysorrheum,2000-03-16,1,Chris Artuoso,Panti Forest,"1 feeding on a low bush, Panti Forest (Bunker ...",These sightings are reproduced with permission...


In [159]:
### COPY TO DRIVE

% cp "/content/df (10).csv" -r "/content/gdrive/My Drive/"
% cp "/content/output.csv" -r "/content/gdrive/My Drive/"
% cp "/content/cachedeng.pkl" -r "/content/gdrive/My Drive/"
% cp "/content/cachedsci.pkl" -r "/content/gdrive/My Drive/"
% cp "/content/contributors.pkl" -r "/content/gdrive/My Drive/"
% cp "/content/cachedpicks.pkl" -r "/content/gdrive/My Drive/"